In [1]:
import seaborn as sns
import pandas as pd
import os
import numpy as np
import math
from math import cos, sin, tanh

In [13]:
#Separating relavent and inrevelant part of the data from the raw txt files.

#--- Insert here ---
pp_id = '09'
position_of = 'Human'
run = True
#-------------------

if run == True:
    # Define input and output file path
    rawfile = open('/Users/nanap/My project/Assets/TrackerData/'+ pp_id + position_of +'_position.txt', 'r')
    Lines = rawfile.readlines()

    path = './newdata/id_' + pp_id + '/' + position_of

    isExist = os.path.exists(path)
    isExist_pp =os.path.exists('./newdata/id_' + pp_id)

    if isExist == False:
        if isExist_pp== False:
            os.mkdir('./newdata/id_' + pp_id)
        os.mkdir(path)

    # Initialize count
    count = 1
    addline = False

    # Read the text file line by line 
    for line in Lines:
        print(line)
        if line == '--- Calibration ---\n':
            print('calibration')

        elif line == '--- Start ' + str(count) + ' ---\n':
            print('START')
            # Create new file with Start 
            newfile = open(path + '/' + pp_id + '_' + str(count) +'.txt', 'w')
            addline = True

        elif line == '--- End ' + str(count) + ' ---\n':
            print("END")
            addline = False
            newfile.close()
            count += 1
        else:    
            if addline==True:
                line = line.replace(",", "") # remove ',' between x and z 
                newfile.write(line) # add valid line to new txt file

            else:
                pass

    rawfile.close()

01:33:44:09 3.943195, 5.535339

01:33:44:19 3.943973, 5.532975

01:33:44:29 3.942688, 5.532021

01:33:44:40 3.93795, 5.53399

01:33:44:50 3.936143, 5.53373

01:33:44:60 3.931544, 5.535114

01:33:44:71 3.925184, 5.537689

01:33:44:81 3.919673, 5.53952

01:33:44:92 3.920636, 5.53707

01:33:45:01 3.927477, 5.530791

01:33:45:11 3.919707, 5.532025

01:33:45:21 3.918988, 5.531045

01:33:45:31 3.920617, 5.526967

01:33:45:42 3.921296, 5.523576

01:33:45:52 3.922081, 5.520775

01:33:45:59 3.922217, 5.519582

01:33:45:72 3.917994, 5.519912

01:33:45:82 3.915587, 5.520324

01:33:45:89 3.914443, 5.520783

01:33:46:02 3.907602, 5.524974

01:33:46:10 3.904172, 5.527395

01:33:46:19 3.903032, 5.528909

01:33:46:29 3.903836, 5.529172

01:33:46:40 3.902526, 5.530982

01:33:46:50 3.901858, 5.533141

01:33:46:61 3.901135, 5.53512

01:33:46:70 3.899886, 5.537759

01:33:46:80 3.899085, 5.539847

01:33:46:91 3.899164, 5.540719

01:33:47:01 3.902369, 5.539423

01:33:47:12 3.903225, 5.539752

01:33:47:21 3.

In [2]:
# Make a function for standard formatting of the dataframe

def format_df(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'], format = '%H:%M:%S:%f')
    df['time'] = df.index/10
    df.iloc[:200, :]
    return df

In [11]:
# Transform coordinate 
# idea: even number human start as 0
#       odd number human start as goal 
# return transformation 

def get_transform(pp_id):
    start_x = []
    start_z = []
    goal_x = []
    goal_z = []
    
    # Adjust index (pp 1~10 did fronting first, pp 11~ did crossing first)
    if int(pp_id)<=10 or int(pp_id) == 21:
        init_idx = 0
    else:
        init_idx = 16
    
    for i in range (1,17):
        # Read each file and add start point to the list 
        position_of = 'Human'
        filepath = './newdata/id_' + pp_id + '/' + position_of + '/' + pp_id + '_' + str(init_idx + i) + '.txt'
        df = pd.read_csv(filepath, sep=" ", header=None, names=["timestamp", "x", "z"])
        
        # Check if the motion tracker was not lost 
        if (df.iloc[0]['x'] == df.iloc[1]['x']):
            print("--- Check manually, motion tracker might have been lost ---")
            print(init_idx + i)
            
        else:
            print("pp_id", pp_id)
            print(filepath)
            if i%2 == 0:
                print(df.iloc[0]['x'])
                start_x.append(df.iloc[0]['x'])
                start_z.append(df.iloc[0]['z'])
        
            else: 
                
                goal_x.append(df.iloc[0]['x'])
                goal_z.append(df.iloc[0]['z'])

    print("Start and goal points", start_x, "\n", start_z , "\n", goal_x, "\n", goal_z)  
                
    # Get the start and goal coordinate by averaging the points
    start_coord = (np.mean(start_x), np.mean(start_z))
    goal_coord = (np.mean(goal_x), np.mean(goal_z))
    print("Start coordinate", start_coord)
    print("Goal coordinate", goal_coord)
    
    distance = math.dist(start_coord, goal_coord)
    
    rotation = np.tanh((goal_coord[1]-start_coord[1])/(goal_coord[0]-start_coord[0]) )
    print("original rotation", rotation*180/np.pi)
    if (goal_coord[0]-start_coord[0])<0 and (goal_coord[1]-start_coord[1])>0:
        rotation += np.pi
    elif (goal_coord[0]-start_coord[0])<0 and (goal_coord[1]-start_coord[1])<0:
        rotation += np.pi
#     elif (goal_coord[0]-start_coord[0])<0 and (goal_coord[1]-start_coord[1])<0:
#         rotation += np.pi*3/2
        
    print("distance [m]: ", distance)
    print("rotation [deg]: ", rotation*180/np.pi)

    return start_coord, rotation*(-1)


In [4]:
def apply_transformation(pp_id, start_coord, rotation, position_of):
    
    # Check if the path to save file exists, if not make one
    path = './traj_data/id_' + pp_id + '/' + position_of

    isExist = os.path.exists(path)
    isExist_pp =os.path.exists('./traj_data/id_' + pp_id)

    if isExist == False:
        if isExist_pp== False:
            os.mkdir('./traj_data/id_' + pp_id)
        os.mkdir(path)
    
    ## Transform the human's start point to origin
    # Read each file 
    for i in range(1,33):
        print("--------", i)
        filepath = './newdata/id_' + pp_id + '/' + position_of + '/' + pp_id + '_' + str(i) + '.txt'
        df = pd.read_csv(filepath, sep=" ", header=None, names=["timestamp", "x", "z"])
        
        df['x'] -= start_coord[0]
        df['z'] -= start_coord[1]
        
        # Apply rotation to follow along x axis
        df['x_new'] = cos(rotation)*df['x'] - sin(rotation)*df['z']
        df['z_new'] = sin(rotation)*df['x'] + cos(rotation)*df['z']
        
        # Now, rotate 180 degrees if it's an even number trials, and shift along x axis (= Spot always start from x=0)
        if i%2 == 0:
            df['x_new2'] = cos(math.pi)*df['x_new'] - sin(math.pi)*df['z_new'] + df['x_new'].max()
            df['z_new2'] = sin(math.pi)*df['x_new'] + cos(math.pi)*df['z_new']
        
        else:
            df['x_new2'] = df['x_new']
            df['z_new2'] = df['z_new']
        
        df['x'] = df['x_new2']
        df['z'] = df['z_new2']
        
        # Remove duplicate
        df = df.drop(['x_new', 'z_new', 'x_new2', 'z_new2'], axis = 1)
        
        # Apply formatting before writing out
        df = format_df(df)
        
        # Write it to csv file (traj_data folder for temporal data)
        df.to_csv('./traj_data/id_' + pp_id + '/' + position_of + '/' + pp_id + '_' + str(i) + '.csv')
        
        print(df)

In [12]:
# Transform coordinates for single participant (for debugging)
#---
pp_id = '11'
position_of = 'Human'
#--- 

start_coord, rotation = get_transform(pp_id)
print(start_coord)
apply_transformation(pp_id, start_coord, rotation, position_of)


--- Check manually, motion tracker might have been lost ---
17
pp_id 11
./newdata/id_11/Human/11_18.txt
1.412648
pp_id 11
./newdata/id_11/Human/11_19.txt
pp_id 11
./newdata/id_11/Human/11_20.txt
1.629177
pp_id 11
./newdata/id_11/Human/11_21.txt
pp_id 11
./newdata/id_11/Human/11_22.txt
1.69153
pp_id 11
./newdata/id_11/Human/11_23.txt
pp_id 11
./newdata/id_11/Human/11_24.txt
1.662419
pp_id 11
./newdata/id_11/Human/11_25.txt
pp_id 11
./newdata/id_11/Human/11_26.txt
1.645
pp_id 11
./newdata/id_11/Human/11_27.txt
pp_id 11
./newdata/id_11/Human/11_28.txt
1.626036
pp_id 11
./newdata/id_11/Human/11_29.txt
pp_id 11
./newdata/id_11/Human/11_30.txt
1.661581
pp_id 11
./newdata/id_11/Human/11_31.txt
pp_id 11
./newdata/id_11/Human/11_32.txt
1.659846
Start and goal points [1.412648, 1.629177, 1.69153, 1.662419, 1.645, 1.626036, 1.661581, 1.659846] 
 [-0.03748325, 0.09760862, 0.09541443, 0.007914219, 0.09076517, 0.08355241, -0.04418204, 0.1540397] 
 [-0.6758795, -0.6475825, -0.5414047, -0.5212267, -0.

                  timestamp         x         z  time
0   1900-01-01 01:39:13.620  3.788804  1.913605   0.0
1   1900-01-01 01:39:13.730  3.787232  1.910739   0.1
2   1900-01-01 01:39:13.840  3.784351  1.908314   0.2
3   1900-01-01 01:39:13.900  3.782275  1.906346   0.3
4   1900-01-01 01:39:14.020  3.776810  1.902915   0.4
..                      ...       ...       ...   ...
140 1900-01-01 01:39:27.630  2.485145 -3.279892  14.0
141 1900-01-01 01:39:27.730  2.487958 -3.277986  14.1
142 1900-01-01 01:39:27.800  2.489982 -3.276473  14.2
143 1900-01-01 01:39:27.900  2.495657 -3.276753  14.3
144 1900-01-01 01:39:28.010  2.502141 -3.279517  14.4

[145 rows x 4 columns]
-------- 16
                  timestamp         x         z  time
0   1900-01-01 01:39:36.210  1.505937  3.247821   0.0
1   1900-01-01 01:39:36.310  1.502246  3.246997   0.1
2   1900-01-01 01:39:36.420  1.499759  3.246478   0.2
3   1900-01-01 01:39:36.520  1.498130  3.246900   0.3
4   1900-01-01 01:39:36.630  1.496842  3.24685

In [6]:
# Transform coordinate for all participants

pp_ids = ["%02d" % x for x in range(1,22)]
positions_of = ["Human", "Spot"]

for pp_id in pp_ids:
    print("===== ", pp_id , "======")
    start_coord, rotation = get_transform(pp_id)
    for name in positions_of:
        apply_transformation(pp_id, start_coord, rotation, name)

=====  11 ======
--- Check manually, motion tracker might have been lost ---
17
Start and goal points [1.412648, 1.629177, 1.69153, 1.662419, 1.645, 1.626036, 1.661581, 1.659846] 
 [-0.03748325, 0.09760862, 0.09541443, 0.007914219, 0.09076517, 0.08355241, -0.04418204, 0.1540397] 
 [-0.6758795, -0.6475825, -0.5414047, -0.5212267, -0.644891, -0.6056163, -0.4074202] 
 [6.018329, 6.084035, 6.084356, 6.15949, 6.127978, 6.086178, 6.167155]
Start coordinate (1.6235296250000002, 0.05595365737499999)
Goal coordinate (-0.5777172714285713, 6.103931571428572)
original rotation -56.82706987537741
distance [m]:  6.4361109956181
rotation [deg]:  123.17293012462258
-------- 1
                  timestamp         x         z  time
0   1900-01-01 01:32:50.610  3.786483  1.806713   0.0
1   1900-01-01 01:32:50.720  3.779878  1.806615   0.1
2   1900-01-01 01:32:50.830  3.779878  1.806615   0.2
3   1900-01-01 01:32:50.930  3.779878  1.806615   0.3
4   1900-01-01 01:32:51.000  3.779878  1.806615   0.4
..     

                  timestamp         x         z  time
0   1900-01-01 01:39:13.620  3.788804  1.913605   0.0
1   1900-01-01 01:39:13.730  3.787232  1.910739   0.1
2   1900-01-01 01:39:13.840  3.784351  1.908314   0.2
3   1900-01-01 01:39:13.900  3.782275  1.906346   0.3
4   1900-01-01 01:39:14.020  3.776810  1.902915   0.4
..                      ...       ...       ...   ...
140 1900-01-01 01:39:27.630  2.485145 -3.279892  14.0
141 1900-01-01 01:39:27.730  2.487958 -3.277986  14.1
142 1900-01-01 01:39:27.800  2.489982 -3.276473  14.2
143 1900-01-01 01:39:27.900  2.495657 -3.276753  14.3
144 1900-01-01 01:39:28.010  2.502141 -3.279517  14.4

[145 rows x 4 columns]
-------- 16
                  timestamp         x         z  time
0   1900-01-01 01:39:36.210  1.505937  3.247821   0.0
1   1900-01-01 01:39:36.310  1.502246  3.246997   0.1
2   1900-01-01 01:39:36.420  1.499759  3.246478   0.2
3   1900-01-01 01:39:36.520  1.498130  3.246900   0.3
4   1900-01-01 01:39:36.630  1.496842  3.24685

-------- 1
                  timestamp         x         z  time
0   1900-01-01 01:32:50.610  0.395903  0.185930   0.0
1   1900-01-01 01:32:50.720  0.395901  0.185677   0.1
2   1900-01-01 01:32:50.830  0.405635  0.186299   0.2
3   1900-01-01 01:32:50.930  0.425443  0.186476   0.3
4   1900-01-01 01:32:51.010  0.446667  0.181497   0.4
..                      ...       ...       ...   ...
158 1900-01-01 01:33:06.430  6.857560  1.209595  15.8
159 1900-01-01 01:33:06.530  6.857427  1.209196  15.9
160 1900-01-01 01:33:06.610  6.858078  1.208269  16.0
161 1900-01-01 01:33:06.710  6.859322  1.207284  16.1
162 1900-01-01 01:33:06.820  6.867070  1.202103  16.2

[163 rows x 4 columns]
-------- 2
                  timestamp         x         z  time
0   1900-01-01 01:33:16.710  0.011469  1.602089   0.0
1   1900-01-01 01:33:16.810  0.009986  1.602654   0.1
2   1900-01-01 01:33:16.920  0.005753  1.604069   0.2
3   1900-01-01 01:33:17.030  0.002256  1.606009   0.3
4   1900-01-01 01:33:17.130  0.00171

                  timestamp         x         z  time
0   1900-01-01 01:46:25.780  0.609955 -0.066355   0.0
1   1900-01-01 01:46:25.850  0.611915 -0.066408   0.1
2   1900-01-01 01:46:25.960  0.627498 -0.062539   0.2
3   1900-01-01 01:46:26.060  0.653599 -0.065454   0.3
4   1900-01-01 01:46:26.170  0.694861 -0.068759   0.4
..                      ...       ...       ...   ...
162 1900-01-01 01:46:41.980  6.395317 -1.744556  16.2
163 1900-01-01 01:46:42.050  6.394851 -1.744551  16.3
164 1900-01-01 01:46:42.160  6.394189 -1.745242  16.4
165 1900-01-01 01:46:42.260  6.394185 -1.744812  16.5
166 1900-01-01 01:46:42.370  6.393897 -1.745062  16.6

[167 rows x 4 columns]
-------- 30
                  timestamp         x         z  time
0   1900-01-01 01:46:50.160  0.012431  1.707061   0.0
1   1900-01-01 01:46:50.260  0.012517  1.707060   0.1
2   1900-01-01 01:46:50.370  0.011015  1.706919   0.2
3   1900-01-01 01:46:50.470  0.006995  1.707867   0.3
4   1900-01-01 01:46:50.580  0.003324  1.70843

                  timestamp         x         z  time
0   1900-01-01 02:17:41.420  0.005554 -1.452851   0.0
1   1900-01-01 02:17:41.530  0.005491 -1.453168   0.1
2   1900-01-01 02:17:41.630  0.003336 -1.452960   0.2
3   1900-01-01 02:17:41.750  0.002622 -1.453051   0.3
4   1900-01-01 02:17:41.810  0.002522 -1.452609   0.4
..                      ...       ...       ...   ...
147 1900-01-01 02:17:56.120  1.334898  3.298468  14.7
148 1900-01-01 02:17:56.230  1.339091  3.296970  14.8
149 1900-01-01 02:17:56.340  1.346704  3.295015  14.9
150 1900-01-01 02:17:56.420  1.354039  3.293462  15.0
151 1900-01-01 02:17:56.520  1.365247  3.290530  15.1

[152 rows x 4 columns]
-------- 17
                  timestamp         x         z  time
0   1900-01-01 02:19:05.730  6.600735 -1.772595   0.0
1   1900-01-01 02:19:05.840  6.599779 -1.770985   0.1
2   1900-01-01 02:19:05.950  6.596550 -1.769770   0.2
3   1900-01-01 02:19:06.020  6.594472 -1.768089   0.3
4   1900-01-01 02:19:06.130  6.592946 -1.76154

...

                  timestamp         x         z  time
0   1900-01-01 03:21:49.960  0.127242 -0.075133   0.0
1   1900-01-01 03:21:50.060  0.127850 -0.074631   0.1
2   1900-01-01 03:21:50.160  0.125697 -0.076782   0.2
3   1900-01-01 03:21:50.270  0.124759 -0.072815   0.3
4   1900-01-01 03:21:50.370  0.123017 -0.072790   0.4
..                      ...       ...       ...   ...
181 1900-01-01 03:22:08.080  6.036115 -0.069774  18.1
182 1900-01-01 03:22:08.150  6.032193 -0.072587  18.2
183 1900-01-01 03:22:08.250  6.038335 -0.076636  18.3
184 1900-01-01 03:22:08.360  6.034766 -0.082750  18.4
185 1900-01-01 03:22:08.470  6.044372 -0.084494  18.5

[186 rows x 4 columns]
-------- 22
                  timestamp         x         z  time
0   1900-01-01 03:22:16.560  0.008154 -0.060709   0.0
1   1900-01-01 03:22:16.660  0.008301 -0.060522   0.1
2   1900-01-01 03:22:16.770  0.006887 -0.059782   0.2
3   1900-01-01 03:22:16.870  0.004956 -0.060737   0.3
4   1900-01-01 03:22:16.950  0.003501 -0.06102

                  timestamp         x         z  time
0   1900-01-01 03:11:04.940  6.406328  0.125832   0.0
1   1900-01-01 03:11:05.030  6.407704  0.124879   0.1
2   1900-01-01 03:11:05.140  6.408640  0.123996   0.2
3   1900-01-01 03:11:05.250  6.409105  0.123789   0.3
4   1900-01-01 03:11:05.350  6.409787  0.122366   0.4
..                      ...       ...       ...   ...
148 1900-01-01 03:11:19.750  0.043625 -0.064189  14.8
149 1900-01-01 03:11:19.860  0.044182 -0.065519  14.9
150 1900-01-01 03:11:19.930  0.044631 -0.066050  15.0
151 1900-01-01 03:11:20.030  0.045366 -0.067357  15.1
152 1900-01-01 03:11:20.150  0.046264 -0.067817  15.2

[153 rows x 4 columns]
-------- 4
                  timestamp         x         z  time
0   1900-01-01 03:11:27.650  6.462182  0.045899   0.0
1   1900-01-01 03:11:27.760  6.460996  0.046204   0.1
2   1900-01-01 03:11:27.820  6.460237  0.046299   0.2
3   1900-01-01 03:11:27.940  6.458874  0.045826   0.3
4   1900-01-01 03:11:28.040  6.457383  0.045606

                  timestamp         x         z  time
0   1900-01-01 03:18:15.650  3.346605  2.523346   0.0
1   1900-01-01 03:18:15.730  3.347031  2.523456   0.1
2   1900-01-01 03:18:15.830  3.348123  2.523708   0.2
3   1900-01-01 03:18:15.930  3.349467  2.523724   0.3
4   1900-01-01 03:18:16.040  3.350064  2.523806   0.4
..                      ...       ...       ...   ...
125 1900-01-01 03:18:28.150  3.400268 -2.492955  12.5
126 1900-01-01 03:18:28.250  3.400758 -2.493117  12.6
127 1900-01-01 03:18:28.320  3.396270 -2.487444  12.7
128 1900-01-01 03:18:28.430  3.394165 -2.487007  12.8
129 1900-01-01 03:18:28.540  3.386683 -2.485312  12.9

[130 rows x 4 columns]
-------- 18
                  timestamp             x         z  time
0   1900-01-01 03:18:39.150  4.440892e-16  2.484111   0.0
1   1900-01-01 03:18:39.260  4.440892e-16  2.484111   0.1
2   1900-01-01 03:18:39.330  4.440892e-16  2.484111   0.2
3   1900-01-01 03:18:39.440  4.440892e-16  2.484111   0.3
4   1900-01-01 03:18:39.54

                  timestamp         x         z  time
0   1900-01-01 03:14:05.550  0.000000 -0.015158   0.0
1   1900-01-01 03:14:05.650  0.000508 -0.015514   0.1
2   1900-01-01 03:14:05.720  0.009649 -0.008892   0.2
3   1900-01-01 03:14:05.830  0.027368 -0.006479   0.3
4   1900-01-01 03:14:05.940  0.066554 -0.001601   0.4
..                      ...       ...       ...   ...
149 1900-01-01 03:14:20.440  5.896798 -0.263035  14.9
150 1900-01-01 03:14:20.540  5.905432 -0.259271  15.0
151 1900-01-01 03:14:20.650  5.914916 -0.256391  15.1
152 1900-01-01 03:14:20.760  5.918886 -0.254335  15.2
153 1900-01-01 03:14:20.830  5.922268 -0.252612  15.3

[154 rows x 4 columns]
-------- 11
                  timestamp         x         z  time
0   1900-01-01 03:14:27.430  0.651688  0.068484   0.0
1   1900-01-01 03:14:27.540  0.651688  0.068484   0.1
2   1900-01-01 03:14:27.650  0.651688  0.068484   0.2
3   1900-01-01 03:14:27.750  0.651688  0.068484   0.3
4   1900-01-01 03:14:27.820  0.651688  0.06848

                  timestamp         x         z  time
0   1900-01-01 03:20:44.340  0.319268 -0.100243   0.0
1   1900-01-01 03:20:44.460  0.319402 -0.099768   0.1
2   1900-01-01 03:20:44.560  0.325134 -0.097324   0.2
3   1900-01-01 03:20:44.630  0.336944 -0.094341   0.3
4   1900-01-01 03:20:44.730  0.363096 -0.089608   0.4
..                      ...       ...       ...   ...
162 1900-01-01 03:21:00.550  6.418005 -0.082371  16.2
163 1900-01-01 03:21:00.650  6.418070 -0.083438  16.3
164 1900-01-01 03:21:00.730  6.418314 -0.081828  16.4
165 1900-01-01 03:21:00.830  6.417792 -0.082389  16.5
166 1900-01-01 03:21:00.940  6.418162 -0.082143  16.6

[167 rows x 4 columns]
-------- 24
                  timestamp         x         z  time
0   1900-01-01 03:21:10.640  0.013078  0.018316   0.0
1   1900-01-01 03:21:10.750  0.013147  0.018698   0.1
2   1900-01-01 03:21:10.850  0.012042  0.020459   0.2
3   1900-01-01 03:21:10.920  0.009404  0.019863   0.3
4   1900-01-01 03:21:11.030  0.004835  0.01917

In [ ]:
pp_id = '02'
position_of = 'Spot'

for i in range (1, 17):
    df_traj = pd.read_csv('./traj_data/id_' + pp_id + '/' + position_of + '/' + pp_id + '_' + str(i) + '.csv')
    print(df_traj)
    
    df_traj = format_df(df_traj)
    sns.scatterplot(data = df_traj.loc[: , ['x', 'z', 'time']], x = 'x', y = 'z', hue = 'time' )
    sns.lineplot(data = df_traj.loc[: , ['x', 'z', 'time']], x = 'x', y = 'z')
